In [5]:
import requests
import nest
import pprint

In [2]:
response = requests.get('http://localhost:5005')
data = response.json()
print(data)

{'nest': 'master@01f6e5a76', 'nestml': '5.0.0-rc1'}


In [6]:
models = ['my_neuron1', 'my_neuron2', 'my_neuron3'][:2]
data = {'models': [{'name': model, 'script': open("my_neuron.nestml_template").read().format(name=model)} for model in models]}
pprint.pprint(data)

{'models': [{'name': 'my_neuron1',
             'script': 'neuron my_neuron1:\n'
                       '\n'
                       '  state:\n'
                       '    r integer = 0                 # counts number of '
                       'tick during the refractory period\n'
                       '    V_abs mV = 0 mV\n'
                       '  end\n'
                       '\n'
                       '  equations:\n'
                       '    kernel I_kernel_inh = (e / tau_syn_inh) * t * '
                       'exp(-t / tau_syn_inh)\n'
                       '    kernel I_kernel_exc = (e / tau_syn_exc) * t * '
                       'exp(-t / tau_syn_exc)\n'
                       '    recordable inline V_m mV = V_abs + E_L # Membrane '
                       'potential.\n'
                       '    inline I pA = convolve(I_kernel_inh, inh_spikes) + '
                       'convolve(I_kernel_exc, exc_spikes) + I_e + I_stim\n'
                       "    V_abs' = -V_a

In [6]:
response = requests.post('http://localhost:5005/install', json=data)

try:
    pprint.pprint(response.json())
except:
    print(response.text)

{'status': {'BUILT': ['nestmlmodule', 'my_neuron1', 'my_neuron2'],
            'INITIALIZED': ['my_neuron1', 'my_neuron2'],
            'INSTALLED': [],
            'WRITTEN': ['my_neuron2', 'my_neuron1']}}


In [2]:
import nest
nest.ResetKernel()
try:
    nest.Install('nestmlmodule')
except:
    pass


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: UNKNOWN
 Built: Mar 30 2022 15:28:56

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.


Apr 12 10:20:54 Install [Info]: 
    loaded module nestmlmodule


In [3]:
models = nest.node_models
my_models = [model for model in models if 'my' in model]
print(my_models)

['my_neuron2', 'my_neuron1']


In [ ]:
data = {'models': [{'name': model_name, 'script': script}]}
data